# Packages and Functions

In [1]:
#We will customize the _numpyarraytolatex function from the array-to-latex library.
#Uncomment the below syntax to install array-to-latex if required.
#!pip install array-to-latex

  Created wheel for clipboard: filename=clipboard-0.0.4-py3-none-any.whl size=1850 sha256=05060c748807cb5b517b1eb2cf2a15296e255dd158adc5dc54d8782103701fca
  Stored in directory: c:\users\sambi\appdata\local\pip\cache\wheels\58\b3\11\d2a638f07f2b7f9210a1a1e8fd621005a836fa1dbf426a1553
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=4260d2bfaaa4fb9a3c564ab4341a56bb7b403ca22bac302e3333af03949adfc2
  Stored in directory: c:\users\sambi\appdata\local\pip\cache\wheels\0c\09\9e\49e21a6840ef7955b06d47394afef0058f0378c0914e48b8b8
Successfully built clipboard pyperclip


In [207]:
def to_scientific_notation(number,digits=2):
    fm1='{:.'+str(digits-1)+'e}'
    fm2='{:.'+str(digits)+'f}e{:+0'+str(digits+1)+'d}'
    a, b = fm1.format(number).split('e')
    return fm2.format(float(a)/10, int(b)+1)

to_scientific_notation(113.2,2)

'0.11e+03'

In [208]:
import numpy as _np
import pandas as _pd

def _numpyarraytolatex(a, frmt=to_scientific_notation, digits=2, arraytype='bmatrix', nargout=0,
                       imstring='j', row=True, mathform=True):
    r"""Return a LaTeX array given a numpy array.

    Parameters
    ----------
    a         : float array
    frmt      : function
        python 3 formatter, optional-
        https://mkaz.tech/python-string-format.html
    digits    : integer
    arraytype : string
        latex array type- `bmatrix` default, optional
    imstring : string (optional)
        Character for square root of -1. Usually i or j
    row      : Boolean
        If the array is 1-D, should the output be
            a row (True) or column (False)

    Returns
    -------
    out: str
        LaTeX array

    See Also
    --------
    to_clp

    Examples
    --------
    >>> import numpy as np
    >>> import array_to_latex as a2l
    >>> A = np.array([[1.23456, 23.45678],[456.23, 8.239521]])
    >>> a2l.to_ltx(A, frmt = '{:6.2f}', arraytype = 'array')
    \begin{array}{c, c}
        1.23 &   23.46\\
      456.23 &    8.24
    \end{array}
    None
    >>> a2l.to_ltx(A, frmt = '{:6.2e}', arraytype = 'array')
    \begin{array}{c, c}
      1.23e+00 &  2.35e+01\\
      4.56e+02 &  8.24e+00
    \end{array}
    None
    >>> a2l.to_ltx(A, frmt = '{:.3g}', arraytype = 'array')
    \begin{array}{c, c}
      1.23 &  23.5\\
      456 &  8.24
    \end{array}
    None
    >>> a2l.to_ltx(A, frmt = '{:1.2f}', arraytype = 'coords')
    {(1.23,23.46),(456.23,8.24)}
    None

    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')

    if len(a.shape) == 1:
        a = _np.array([a])
        if row is False:
            a = a.T

    if arraytype == "coords":
        coords = ['(' + ','.join([frmt(x,digits) for x in r]) + ')' for r in a]
        return '{' + ','.join(coords) + '}'

    arrayformat = ''

    if arraytype == 'array':
        arrayformat = '{'
        for _ in _np.arange(a.shape[1]):
            arrayformat = arrayformat + ' c,'
        arrayformat = arrayformat[:-1] + '}'

    out = r'\begin{' + arraytype + '}' + arrayformat + '\n'
    for i in _np.arange(a.shape[0]):
        out = out + ' '
        for j in _np.arange(a.shape[1]):
            leadstr = '' if _np.real(a[i, j]) < 0 else ' '
            dot_space = ' ' if '.' not in frmt(a[i, j],digits) else ''
            if _np.iscomplexobj(a[i, j]):
                out = (out + leadstr
                       + math_form(frmt(_np.real(a[i, j]),digits),
                                   mathform=mathform)
                       + ' + '
                       + math_form(frmt(_np.imag(a[i, j]),digits),
                                   is_imaginary=True,
                                   mathform=mathform)
                       + imstring
                       + dot_space + ' & ')
            else:
                out = (out
                       + leadstr
                       + math_form(frmt(_np.real(a[i, j]),digits),
                                   mathform=mathform)
                       + dot_space
                       + r' & ')

        out = out[:-3]
        out = out + '\\\\\n'

    out = out[:-3] + '\n' + r'\end{' + arraytype + '}'

    return out

def to_ltx(a, frmt=to_scientific_notation, digits=2, arraytype=None, nargout=0,
           imstring='j', row=True, mathform=True, print_out=True):
    r"""
    Print or return a LaTeX array given a numpy array or Pandas dataframe.

    Parameters
    ----------
    a         : float array
    frmt      : string
        python 3 formatter, optional-
        https://mkaz.tech/python-string-format.html
    digits    : integer
    arraytype : string
        latex array type- `bmatrix` default, optional
    imstring : string (optional)
        Character for square root of -1. Usually i or j
    row        : Boolean (optional: default True)
        If the array is 1-D, should the output be
            a row (True) or column (False)
    mathform  : Boolean (optional: default True)
        Replace #E# with #\times10^{#}


    Returns
    -------
    out: str
        LaTeX array

    See Also
    --------
    to_clp

    Examples
    --------
    >>> import numpy as np
    >>> import array_to_latex as a2l
    >>> A = np.array([[1.23456, 23.45678],[456.23, 8.239521]])
    >>> a2l.to_ltx(A, frmt = '{:6.2f}', arraytype = 'array')
    \begin{array}
        1.23 &   23.46\\
      456.23 &    8.24
    \end{array}
    None
    >>> a2l.to_ltx(A, frmt = '{:6.2e}', arraytype = 'array')
    \begin{array}
      1.23e+00 &  2.35e+01\\
      4.56e+02 &  8.24e+00
    \end{array}
    None
    >>> a2l.to_ltx(A, frmt = '{:.3g}', arraytype = 'array')
    \begin{array}
      1.23 &  23.5\\
      456  &  8.24
    \end{array}
    None
    >>> a2l.to_ltx(A, frmt = '{:1.2f}', arraytype = 'coords')
    {(1.23,23.46),(456.23,8.24)}
    None

    """
    if isinstance(a, _np.ndarray):

        if arraytype is None:
            arraytype = 'bmatrix'
        latex = _numpyarraytolatex(a, frmt=frmt, digits=digits, arraytype=arraytype,
                                   nargout=nargout, imstring=imstring,
                                   row=row, mathform=mathform)
    else:
        raise TypeError("Argument should be a "
                        "numpy array.")
    if print_out is True:
        print(latex)
        return

    return latex

def math_form(number, is_imaginary=False, mathform=True):
    if 'e' in number:
        if mathform:
            number = number.replace('e', '\\times 10^{') + '}'
        else:
            number = number.replace('e', '\\mathrm{e}{') + '}'
    return number

##### Relaxation Method

In [252]:
import numpy as np

# Relaxation method
def relaxation_method(A, b, x0, tolerance=0.001, max_iterations=1000, omega=1.25):
    """
    Solve a system of linear equations using the Relaxation method.

    Arguments:
    A: 2D array representing the coefficients of the linear equations
    b: 1D array representing the constants on the right-hand side of the equations
    x0: 1D array representing the initial guess for the solution
    tolerance: Desired tolerance for the solution
    max_iterations: Maximum number of iterations allowed
    omega: Relaxation parameter (typically between 1 and 2)

    Returns:
    solution: 1D array representing the solution to the system of linear equations
    num_iterations: Number of iterations performed
    """
    n = len(b)
    solution = np.copy(x0)
    for iteration in range(max_iterations):
        old_solution = np.copy(solution)
        for i in range(n):
            summation = 0.0
            for j in range(n):
                if j != i:
                    summation=summation+A[i][j] * solution[j]
            solution[i] = (1 - omega) * old_solution[i] + (omega / A[i][i]) * (b[i] - summation)
        if np.linalg.norm(solution - old_solution) < tolerance:
            return solution, iteration + 1
    print("Warning: Maximum number of iterations reached without convergence.")
    return solution, max_iterations

In [ ]:
import numpy as np 
import array_to_latex as a2l 

def gauss_jacobi(A, b, x0, tol=0.01, max_iter=100):
    n = len(A)
    print("A=")
    a2l.to_ltx(A, frmt = '{:6.7f}', arraytype = 'bmatrix')
    print("B=")
    a2l.to_ltx(b, frmt = '{:6.7f}', arraytype = 'bmatrix')
    print("x0=")
    a2l.to_ltx(x0, frmt = '{:6.7f}', arraytype = 'bmatrix')
    U = -np.triu(A,k=1)
    L = -np.tril(A,k=-1)
    D = A + U + L
    T = np.matmul(np.linalg.inv(D),np.add(L,U))
    C = np.matmul(np.linalg.inv(D),b)
    print("D=")
    a2l.to_ltx(D, frmt = '{:6.7f}', arraytype = 'bmatrix')
    print("L=")
    a2l.to_ltx(L, frmt = '{:6.7f}', arraytype = 'bmatrix')
    print("U=")
    a2l.to_ltx(U, frmt = '{:6.7f}', arraytype = 'bmatrix')
    print("T=")
    a2l.to_ltx(T, frmt = '{:6.7f}', arraytype = 'bmatrix')
    print("C=")
    a2l.to_ltx(C, frmt = '{:6.7f}', arraytype = 'bmatrix')
    x = x0[:]
    for k in range(max_iter):
        print("iteration=",k)
        x_old = x
        x = np.add(np.matmul(T,x),C)
        print("x=")
        a2l.to_ltx(x, frmt = '{:6.7f}', arraytype = 'bmatrix')
        if abs(x - x_old).all() < tol:
            return x
    raise ValueError("Gauss-Seidel method did not converge")
    
A = np.array([[4, -1, 0],
     [-1, 4, -1],
     [0, -1, 3]]).astype(float)
b = np.array([5, -10, 15]).astype(float)
x0 = np.array([0, 0, 0]).astype(float)
solution = gauss_jacobi(A, b, x0)
print("Solution:", solution)

In [242]:
import numpy as np
# Conjugate Gradient method with latex output
def conjugate_gradient_L(A, b, x0, digits=2, tol=0.001, max_iter=1000):
    """
    Solve the linear system Ax = b using Conjugate Gradient method.

    Parameters:
        A (numpy.ndarray): Coefficient matrix.
        b (numpy.ndarray): Right-hand side vector.
        x0 (numpy.ndarray): Initial guess for the solution.
        digits (int): Number of digits of decimal computer latex output.
        tol (float): Tolerance for convergence.
        max_iter (int): Maximum number of iterations.

    Returns:
        x (numpy.ndarray): Solution vector.
        num_iter (int): Number of iterations performed.
    """
    def floatformatter(num,digits=digits):
        return to_scientific_notation(num,digits=digits)
    np.set_printoptions(formatter={'float_kind':floatformatter})
    r = b - np.dot(A, x0)
    p = r
    #print("p=")
    #a2l.to_ltx(p, frmt = fm, arraytype = 'bmatrix')
    x = x0
    rsold = np.dot(r, r)
    #print("|r|^2=",rsold)
    for i in range(max_iter):
        print("Iteration",i,":")
        print("r=")
        to_ltx(r, digits=digits, arraytype = 'bmatrix')
        Ap = np.dot(A, p)
        print("Ap=")
        to_ltx(Ap, digits=digits, arraytype = 'bmatrix')
        alpha = rsold / np.dot(p, Ap)
        print("alpha=",to_scientific_notation(alpha,digits))
        x = x + alpha * p
        print("x",i+1,"=")
        to_ltx(x, digits=digits, arraytype = 'bmatrix')
        r = r - alpha * Ap
        print("r=")
        to_ltx(r, digits=digits, arraytype = 'bmatrix')
        rsnew = np.dot(r, r)
        print("|r|^2=",to_scientific_notation(rsnew,digits))
        beta = (rsnew / rsold)
        print("beta",to_scientific_notation(beta,digits))
        if np.sqrt(rsnew) < tol:
            break
        p = r + beta * p
        print("new p=")
        to_ltx(p, digits=digits, arraytype = 'bmatrix')
        rsold = rsnew
    return x, i+1

In [236]:
import numpy as np

def gauss_elim(A, b, digits=2):
    n = len(A)
    
    # Augmenting the matrix
    AMN = np.hstack((A, b.reshape(-1, 1)))
    
    print("A=")
    to_ltx(AMN[0:,0:n], digits=digits, arraytype = 'bmatrix')
    print("B=")
    to_ltx(AMN[0:,n], digits=digits, arraytype = 'bmatrix')
    
    for i in range(n):
        # Partial pivoting
        max_index = np.argmax(np.abs(AMN[i:, i])) + i
        if max_index != i:
            AMN[[i, max_index]] = AMN[[max_index, i]]
            print("After partial pivoting:")
            print("R",i+1,"swap R",max_index+1)
            print("A=")
            to_ltx(AMN[0:,0:n], digits=digits, arraytype = 'bmatrix')
            print("B=")
            to_ltx(AMN[0:,n], digits=digits, arraytype = 'bmatrix')
            
        #Normalization
        print("R",i+1,"= R",i+1,"/",AMN[i,i])
        AMN[i] = AMN[i]/AMN[i,i]
        print("A=")
        to_ltx(AMN[0:,0:n], digits=digits, arraytype = 'bmatrix')
        print("B=")
        to_ltx(AMN[0:,n], digits=digits, arraytype = 'bmatrix')
            
        # Elimination
        for j in range(i+1, n):
            print("R",j+1,"= R",j+1,"-",AMN[j,i],"* R",i+1)
            AMN[j, i:] -= AMN[j, i] * AMN[i, i:]
            print("A=")
            to_ltx(AMN[0:,0:n], digits=digits, arraytype = 'bmatrix')
            print("B=")
            to_ltx(AMN[0:,n], digits=digits, arraytype = 'bmatrix')
                
        # Back substitution
        x = np.zeros(n)
        for i in range(n-1, -1, -1):
            x[i] = (AMN[i, -1] - np.dot(AMN[i, i+1:n], x[i+1:]))
            
    return x

NameError: name 'xt' is not defined

### Question 1

In [247]:
import numpy as np
# Solving Q1 using Gaussian elimination with partial pivoting
A = np.array([[1.00,0.50],[0.50,0.33]])
b = np.array([0.24,0.13])

solution = gauss_elim(A, b,digits=2)
print("Solution:", solution)

A=
\begin{bmatrix}
  0.10\times 10^{+01} &  0.50\times 10^{+00}\\
  0.50\times 10^{+00} &  0.33\times 10^{+00}
\end{bmatrix}
B=
\begin{bmatrix}
  0.24\times 10^{+00} &  0.13\times 10^{+00}
\end{bmatrix}
R 1 = R 1 / 1.0
A=
\begin{bmatrix}
  0.10\times 10^{+01} &  0.50\times 10^{+00}\\
  0.50\times 10^{+00} &  0.33\times 10^{+00}
\end{bmatrix}
B=
\begin{bmatrix}
  0.24\times 10^{+00} &  0.13\times 10^{+00}
\end{bmatrix}
R 2 = R 2 - 0.5 * R 1
A=
\begin{bmatrix}
  0.10\times 10^{+01} &  0.50\times 10^{+00}\\
  0.00\times 10^{+01} &  0.80\times 10^{-01}
\end{bmatrix}
B=
\begin{bmatrix}
  0.24\times 10^{+00} &  0.10\times 10^{-01}
\end{bmatrix}
R 2 = R 2 / 0.08000000000000002
A=
\begin{bmatrix}
  0.10\times 10^{+01} &  0.50\times 10^{+00}\\
  0.00\times 10^{+01} &  0.10\times 10^{+01}
\end{bmatrix}
B=
\begin{bmatrix}
  0.24\times 10^{+00} &  0.13\times 10^{+00}
\end{bmatrix}
Solution: [0.18e+00 0.13e+00]


In [243]:
import numpy as np
# Solving Q1 by Conjugate Gradient method
A = np.array([[1.00,0.50],[0.50,0.33]])
b = np.array([0.24,0.13])
x0 = np.array([0,0])
solution, iterations = conjugate_gradient_L(A, b, x0)
print("Solution:", solution)
print("Number of iterations:", iterations)

Iteration 0 :
r=
\begin{bmatrix}
  0.24\times 10^{+00} &  0.13\times 10^{+00}
\end{bmatrix}
Ap=
\begin{bmatrix}
  0.30\times 10^{+00} &  0.16\times 10^{+00}
\end{bmatrix}
alpha= 0.79e+00
x 1 =
\begin{bmatrix}
  0.19\times 10^{+00} &  0.10\times 10^{+00}
\end{bmatrix}
r=
\begin{bmatrix}
 -0.76\times 10^{-03} &  0.14\times 10^{-02}
\end{bmatrix}
|r|^2= 0.26e-05
beta 0.34e-04
new p=
\begin{bmatrix}
 -0.75\times 10^{-03} &  0.14\times 10^{-02}
\end{bmatrix}
Iteration 1 :
r=
\begin{bmatrix}
 -0.76\times 10^{-03} &  0.14\times 10^{-02}
\end{bmatrix}
Ap=
\begin{bmatrix}
 -0.48\times 10^{-04} &  0.89\times 10^{-04}
\end{bmatrix}
alpha= 0.16e+02
x 2 =
\begin{bmatrix}
  0.18\times 10^{+00} &  0.13\times 10^{+00}
\end{bmatrix}
r=
\begin{bmatrix}
 -0.17\times 10^{-15} & -0.91\times 10^{-16}
\end{bmatrix}
|r|^2= 0.37e-31
beta 0.15e-25
Solution: [0.18e+00 0.13e+00]
Number of iterations: 2


### Question 2

In [246]:
# Solving Q2 using Gaussian elimination with partial pivoting
A = np.array([[0.1,0.2],[0.2,11e1]])
b = np.array([0.3,11e1])

solution = gauss_elim(A, b,digits=2)
print("Solution:", solution)

A=
\begin{bmatrix}
  0.10\times 10^{+00} &  0.20\times 10^{+00}\\
  0.20\times 10^{+00} &  0.11\times 10^{+03}
\end{bmatrix}
B=
\begin{bmatrix}
  0.30\times 10^{+00} &  0.11\times 10^{+03}
\end{bmatrix}
After partial pivoting:
R 1 swap R 2
A=
\begin{bmatrix}
  0.20\times 10^{+00} &  0.11\times 10^{+03}\\
  0.10\times 10^{+00} &  0.20\times 10^{+00}
\end{bmatrix}
B=
\begin{bmatrix}
  0.11\times 10^{+03} &  0.30\times 10^{+00}
\end{bmatrix}
R 1 = R 1 / 0.2
A=
\begin{bmatrix}
  0.10\times 10^{+01} &  0.55\times 10^{+03}\\
  0.10\times 10^{+00} &  0.20\times 10^{+00}
\end{bmatrix}
B=
\begin{bmatrix}
  0.55\times 10^{+03} &  0.30\times 10^{+00}
\end{bmatrix}
R 2 = R 2 - 0.1 * R 1
A=
\begin{bmatrix}
  0.10\times 10^{+01} &  0.55\times 10^{+03}\\
  0.00\times 10^{+01} & -0.55\times 10^{+02}
\end{bmatrix}
B=
\begin{bmatrix}
  0.55\times 10^{+03} & -0.55\times 10^{+02}
\end{bmatrix}
R 2 = R 2 / -54.8
A=
\begin{bmatrix}
  0.10\times 10^{+01} &  0.55\times 10^{+03}\\
  -0.00\times 10^{+01} &  0.1

In [245]:
# Solving Q2 by Conjugate Gradient method
A = np.array([[0.1,0.2],[0.2,11e1]])
b = np.array([0.3,11e1])
x0 = np.array([0,0])
solution, iterations = conjugate_gradient_L(A, b, x0)
print("Solution:", solution)
print("Number of iterations:", iterations)

Iteration 0 :
r=
\begin{bmatrix}
  0.30\times 10^{+00} &  0.11\times 10^{+03}
\end{bmatrix}
Ap=
\begin{bmatrix}
  0.22\times 10^{+02} &  0.12\times 10^{+05}
\end{bmatrix}
alpha= 0.91e-02
x 1 =
\begin{bmatrix}
  0.27\times 10^{-02} &  0.10\times 10^{+01}
\end{bmatrix}
r=
\begin{bmatrix}
  0.10\times 10^{+00} & -0.27\times 10^{-03}
\end{bmatrix}
|r|^2= 0.99e-02
beta 0.82e-06
new p=
\begin{bmatrix}
  0.10\times 10^{+00} & -0.18\times 10^{-03}
\end{bmatrix}
Iteration 1 :
r=
\begin{bmatrix}
  0.10\times 10^{+00} & -0.27\times 10^{-03}
\end{bmatrix}
Ap=
\begin{bmatrix}
  0.99\times 10^{-02} & -0.27\times 10^{-04}
\end{bmatrix}
alpha= 0.10e+02
x 2 =
\begin{bmatrix}
  0.10\times 10^{+01} &  0.10\times 10^{+01}
\end{bmatrix}
r=
\begin{bmatrix}
  0.39\times 10^{-13} &  0.21\times 10^{-10}
\end{bmatrix}
|r|^2= 0.45e-21
beta 0.46e-19
Solution: [0.10e+01 0.10e+01]
Number of iterations: 2


### Question 4

In [262]:
# To check solution of question 4
np.set_printoptions()
A = np.array([[3,-1,1],[3,6,2],[3,3,7]]).astype(float)
b = np.array([1,0,4]).astype(float)
x0 = np.array([0,0,0]).astype(float)
solution, iterations = relaxation_method(A, b, x0, max_iterations=2, omega=1.1)
print("Solution:", solution)
print("Number of iterations:", iterations)

Solution: [ 0.05410079 -0.21154353  0.64771586]
Number of iterations: 2


In [280]:
A = np.array([[10,-1,0],[-1,10,-2],[0,-2,10]]).astype(float)
b = np.array([9,7,6]).astype(float)
x0 = np.array([0,0,0]).astype(float)
solution, iterations = relaxation_method(A, b, x0, max_iterations=2, omega=1.1)
print("Solution:", solution)
print("Number of iterations:", iterations)

Solution: [0.987679   0.97849345 0.78993276]
Number of iterations: 2


In [281]:
A = np.array([[10,5,0,0],[5,10,-4,0],[0,-4,8,-1],[0,0,-3,4]]).astype(float)
b = np.array([6,25,-11,-11]).astype(float)
x0 = np.array([0,0,0,0]).astype(float)
solution, iterations = relaxation_method(A, b, x0, max_iterations=2, omega=1.1)
print("Solution:", solution)
print("Number of iterations:", iterations)

Solution: [-0.71885     2.8188215  -0.38076847 -3.02016286]
Number of iterations: 2


In [283]:
A = np.array([[4,1,1,0,1],[-1,-3,1,1,0],[2,1,5,-1,-1],[-1,-1,-1,4,0],[0,2,-1,1,4]]).astype(float)
b = np.array([6,6,6,6,6]).astype(float)
x0 = np.array([0,0,0,0,0]).astype(float)
solution, iterations = relaxation_method(A, b, x0, max_iterations=2, omega=1.1)
print("Solution:", solution)
print("Number of iterations:", iterations)

Solution: [ 1.07967477 -1.260654    2.04248922  1.9953725   2.0495358 ]
Number of iterations: 2


### Question 5

In [284]:
A = np.array([[3,-1,1],[3,6,2],[3,3,7]]).astype(float)
b = np.array([1,0,4]).astype(float)
x0 = np.array([0,0,0]).astype(float)
solution, iterations = relaxation_method(A, b, x0, max_iterations=2, omega=1.3)
print("Solution:", solution)
print("Number of iterations:", iterations)

Solution: [-0.10401032 -0.13318139  0.67749966]
Number of iterations: 2


In [285]:
A = np.array([[10,-1,0],[-1,10,-2],[0,-2,10]]).astype(float)
b = np.array([9,7,6]).astype(float)
x0 = np.array([0,0,0]).astype(float)
solution, iterations = relaxation_method(A, b, x0, max_iterations=2, omega=1.3)
print("Solution:", solution)
print("Number of iterations:", iterations)

Solution: [0.957073   0.99038745 0.72065694]
Number of iterations: 2


In [287]:
A = np.array([[10,5,0,0],[5,10,-4,0],[0,-4,8,-1],[0,0,-3,4]]).astype(float)
b = np.array([6,25,-11,-11]).astype(float)
x0 = np.array([0,0,0,0]).astype(float)
solution, iterations = relaxation_method(A, b, x0, max_iterations=2, omega=1.3)
print("Solution:", solution)
print("Number of iterations:", iterations)

Solution: [-1.23695     3.2287515  -0.26910492 -2.76354642]
Number of iterations: 2


In [288]:
A = np.array([[4,1,1,0,1],[-1,-3,1,1,0],[2,1,5,-1,-1],[-1,-1,-1,4,0],[0,2,-1,1,4]]).astype(float)
b = np.array([6,6,6,6,6]).astype(float)
x0 = np.array([0,0,0,0,0]).astype(float)
solution, iterations = relaxation_method(A, b, x0, max_iterations=2, omega=1.3)
print("Solution:", solution)
print("Number of iterations:", iterations)

Solution: [ 0.70642575 -0.41038757  2.41706293  2.25195461  1.06150743]
Number of iterations: 2


### Question 13

In [276]:
# Verification for LU decomposition
import array_to_latex as a2l
L1 = np.array([[1,0,0],[1,1,0],[1,0.571,1]])
U1 = np.array([[3 ,-1, 1],[0 ,7, 1],[0,0,5.428]])
A1 = L1@U1
print(a2l.to_ltx(A1, frmt = '{:f}', arraytype = 'bmatrix'))

\begin{bmatrix}
  3.000000 & -1.000000 &  1.000000\\
  3.000000 &  6.000000 &  2.000000\\
  3.000000 &  2.997000 &  6.999000
\end{bmatrix}
None


In [277]:
# Verification for LU decomposition
L1 = np.array([[1,0,0],[-0.1,1,0],[0,-0.202,1]])
U1 = np.array([[10 ,-1, 0],[0 ,9.9, -2],[0,0,9.596]])
A1 = L1@U1
print(a2l.to_ltx(A1, frmt = '{:f}', arraytype = 'bmatrix'))

\begin{bmatrix}
  10.000000 & -1.000000 &  0.000000\\
 -1.000000 &  10.000000 & -2.000000\\
  0.000000 & -1.999800 &  10.000000
\end{bmatrix}
None


In [279]:
L1 = np.array([[1,0,0,0],[0.5,1,0,0],[0,-0.533,1,0],[0, 0, -0.170, 1]])
U1 = np.array([[10 ,5, 0,0],[0 ,7.5, -4,0],[0,0,5.867,-1],[0, 0, 0, 4.83]])
A1 = L1@U1
print(a2l.to_ltx(A1, frmt = '{:f}', arraytype = 'bmatrix'))

\begin{bmatrix}
  10.000000 &  5.000000 &  0.000000 &  0.000000\\
  5.000000 &  10.000000 & -4.000000 &  0.000000\\
  0.000000 & -3.997500 &  7.999000 & -1.000000\\
  0.000000 &  0.000000 & -0.997390 &  5.000000
\end{bmatrix}
None


### Question 16

In [290]:
A=np.array([[0.2,0.1,1,1,0],[0.1,4,-1,1,-1],[1,-1,60,0,-2],[1,1,0,8,4],[0,-1,-2,4,700]]).astype(float)
b=np.array([1,2,3,4,5]).astype(float)
x0=np.array([0,0,0,0,0]).astype(float)
xt = np.array([7.859713071, 0.422926408, -0.073592239, -0.540643016, 0.010626163])

In [291]:
# Jacobi method
def gauss_jacobi(A, b, x0, tol=0.01, max_iter=100):
    n = len(A)
    U = -np.triu(A,k=1)
    L = -np.tril(A,k=-1)
    D = A + U + L
    T = np.matmul(np.linalg.inv(D),np.add(L,U))
    C = np.matmul(np.linalg.inv(D),b)
    x = np.copy(x0)
    for k in range(max_iter):
        x_old = np.copy(x)
        x = np.add(np.matmul(T,x),C)
        if np.linalg.norm(x - xt) < tol:
            return x,k+1
    raise ValueError("Gauss-Jacobi method did not converge")

solution = gauss_jacobi(A, b, x0)
print("Solution:", solution[0])
print("Number of iterations:", solution[1])

Solution: [ 7.86798447  0.4239069  -0.0731638  -0.5370925   0.01063184]
Number of iterations: 39


In [292]:
# Gauss-Seidel method
def gauss_siedel(A, b, x0, tol=0.01, max_iter=1000):
    n = len(A)
    U = -np.triu(A,k=1)
    L = -np.tril(A,k=-1)
    D = A + U + L
    T = np.matmul(np.linalg.inv(np.add(D,-L)),U)
    C = np.matmul(np.linalg.inv(np.add(D,-L)),b)
    x = np.copy(x0)
    for k in range(max_iter):
        x_old = np.copy(x)
        x = np.add(np.matmul(T,x),C)
        if np.linalg.norm(x - xt) < tol:
            return x,k+1
    raise ValueError("Gauss-Seidel method did not converge")

solution = gauss_siedel(A, b, x0)
print("Solution:", solution[0])
print("Number of iterations:", solution[1])

Solution: [ 7.85091478  0.42280131 -0.07344797 -0.53952326  0.01062   ]
Number of iterations: 18


In [293]:
# Relaxation method
def relaxation_method(A, b, x0, tol=0.01, max_iter=1000, omega=1.25):
    n = len(b)
    x = np.copy(x0)
    for k in range(max_iter):
        x_old = np.copy(x)
        for i in range(n):
            sum = 0
            for j in range(n):
                if j != i:
                    sum += A[i][j] * x[j]
            x[i] = (1 - omega) * x_old[i] + (omega / A[i][i]) * (b[i] - sum)
        r = np.multiply(np.diag(A),np.add(x,-x_old))
        if np.linalg.norm(x - xt) < tol:
            return x,k+1
    raise ValueError("Relaxation method did not converge")

solution = relaxation_method(A, b, x0)
print("Solution:", solution[0])
print("Number of iterations:", solution[1])

Solution: [ 7.85152701  0.42277371 -0.07348303 -0.53978369  0.01062286]
Number of iterations: 7


In [294]:
# Conjugate Gradient method
def conjugate_gradient(A, b, x0, tol=0.01, max_iter=1000):
    r = b - np.dot(A, x0)
    p = np.copy(r)
    x = np.copy(x0)
    rsold = np.dot(r, r)
    for k in range(max_iter):
        Ap = np.dot(A, p)
        alpha = rsold / np.dot(p, Ap)
        x = x + alpha * p
        if np.linalg.norm(x - xt) < tol:
            return x,k+1
        r = r - alpha * Ap
        rsnew = np.dot(r, r)
        beta = (rsnew / rsold)
        p = r + beta * p
        rsold = np.copy(rsnew)
    raise ValueError("Relaxation method did not converge")

solution = conjugate_gradient(A, b, x0)
print("Solution:", solution[0])
print("Number of iterations:", solution[1])

Solution: [ 7.85971308  0.42292641 -0.07359224 -0.54064302  0.01062616]
Number of iterations: 5


### Question 17

In [296]:
def qr_eigenvalues(A, max_iter=100, tol=1e-6):
    n = A.shape[0]
    V = np.eye(n)

    for _ in range(max_iter):
        Q, R = np.linalg.qr(A)
        A = np.dot(R, Q)
        V = np.dot(V, Q)
        if np.abs(A.diagonal(-1)).max() < tol:
            break

    eigenvalues = A.diagonal()
    return eigenvalues, V


A=np.array([[5,-2],[-2,8]])
eigenvalues, eigenvectors = qr_eigenvalues(A)
print("Eigenvalues using QR decomposition:",eigenvalues)
eigenvalues1, eigenvectors1=np.linalg.eigh(A)
print("Eigenvalues using numpy.linalg.eigh:",eigenvalues1)

Eigenvalues using QR decomposition: [9. 4.]
Eigenvalues using numpy.linalg.eigh: [4. 9.]


### Question 18

In [295]:
import numpy as np

def power_method(A, max_iter=1000, tol=0.01):
    n = A.shape[0]
    x = np.random.rand(n)
    x /= np.linalg.norm(x)
    ev0 = 0

    for k in range(max_iter):
        x_new = np.matmul(A, x)
        ev = np.dot(x_new, x)
        x_new /= np.linalg.norm(x_new)

        if abs(ev0 - ev)/ev < tol:
            break

        x = x_new
        ev0 = ev

    return ev

# Example matrix
A = np.array([[2,-1,0],[-1,2,-1],[0,-1,2]])

# Apply Power Method to find dominant eigenvalue and eigenvector
eigenvalue = power_method(A)

print("Dominant eigenvalue:", eigenvalue)

Dominant eigenvalue: 3.4066566424862823


### Question 19

In [269]:
import time

def sigma(sv, m, n):
    Sigma = np.zeros((m, n))

    min_dim = min(m, n)
    Sigma[:min_dim, :min_dim] = np.diag(sv)

    return Sigma

In [270]:
st = time.time()
A1=np.array([[2,1],[1,0]])
U1,S1,V1=np.linalg.svd(A1)

Sigma1 = sigma(S1,U1.shape[1],V1.shape[1])

D1=np.dot(np.dot(U1,Sigma1),V1)

et = time.time()
print("Matrix(U):",U1)

print("Matrix(S):",Sigma1)

print("Matrix(V):",V1)

print("Matrix Product(USV):",D1)

print("Time for computation",et-st)

Matrix(U): [[-0.92387953 -0.38268343]
 [-0.38268343  0.92387953]]
Matrix(S): [[2.41421356 0.        ]
 [0.         0.41421356]]
Matrix(V): [[-0.92387953 -0.38268343]
 [ 0.38268343 -0.92387953]]
Matrix Product(USV): [[ 2.00000000e+00  1.00000000e+00]
 [ 1.00000000e+00 -3.81016887e-17]]
Time for computation 0.013469457626342773


In [271]:
st = time.time()
A2=np.array([[2,1],[1,0],[0,1]])
U2,S2,V2=np.linalg.svd(A2)

Sigma2 = sigma(S2,U2.shape[1],V2.shape[1])

D2=np.dot(np.dot(U2,Sigma2),V2)

et = time.time()
print("Matrix(U):",U2)

print("Matrix(S):",Sigma2)

print("Matrix(V):",V2)

print("Matrix Product(USV):",D2)

print("Time for computation",et-st)

Matrix(U): [[-9.12870929e-01  4.80183453e-17 -4.08248290e-01]
 [-3.65148372e-01 -4.47213595e-01  8.16496581e-01]
 [-1.82574186e-01  8.94427191e-01  4.08248290e-01]]
Matrix(S): [[2.44948974 0.        ]
 [0.         1.        ]
 [0.         0.        ]]
Matrix(V): [[-0.89442719 -0.4472136 ]
 [-0.4472136   0.89442719]]
Matrix Product(USV): [[ 2.00000000e+00  1.00000000e+00]
 [ 1.00000000e+00 -6.81060746e-17]
 [-1.23617226e-16  1.00000000e+00]]
Time for computation 0.0010411739349365234


In [272]:
st = time.time()
A3=np.array([[2,1],[-1,1],[1,1],[2,-1]])
U3,S3,V3=np.linalg.svd(A3)

Sigma3 = sigma(S3,U3.shape[1],V3.shape[1])

D3=np.dot(np.dot(U3,Sigma3),V3)

et = time.time()
print("Matrix(U):",U3)

print("Matrix(S):",Sigma3)

print("Matrix(V):",V3)

print("Matrix Product(USV):",D3)

print("Time for computation",et-st)

Matrix(U): [[-0.63245553 -0.5        -0.52229321 -0.27786652]
 [ 0.31622777 -0.5        -0.30196857  0.74753928]
 [-0.31622777 -0.5         0.79704714  0.12130893]
 [-0.63245553  0.5        -0.02721464  0.59098169]]
Matrix(S): [[3.16227766 0.        ]
 [0.         2.        ]
 [0.         0.        ]
 [0.         0.        ]]
Matrix(V): [[-1. -0.]
 [-0. -1.]]
Matrix Product(USV): [[ 2.  1.]
 [-1.  1.]
 [ 1.  1.]
 [ 2. -1.]]
Time for computation 0.0


In [273]:
st = time.time()
A4=np.array([[1,1,0],[-1,0,1],[0,1,-1],[1,1,-1]])
U4,S4,V4=np.linalg.svd(A4)

Sigma4 = sigma(S4,U4.shape[1],V4.shape[1])

D4=np.dot(np.dot(U4,Sigma4),V4)

et = time.time()
print("Matrix(U):",U4)

print("Matrix(S):",Sigma4)

print("Matrix(V):",V4)

print("Matrix Product(USV):",D4)

print("Time for computation",et-st)

Matrix(U): [[-4.36435780e-01  7.07106781e-01  4.08248290e-01 -3.77964473e-01]
 [ 4.36435780e-01  7.07106781e-01 -4.08248290e-01  3.77964473e-01]
 [-4.36435780e-01  3.33066907e-16 -8.16496581e-01 -3.77964473e-01]
 [-6.54653671e-01  3.33066907e-16 -5.55111512e-17  7.55928946e-01]]
Matrix(S): [[2.64575131 0.         0.        ]
 [0.         1.         0.        ]
 [0.         0.         1.        ]
 [0.         0.         0.        ]]
Matrix(V): [[-0.57735027 -0.57735027  0.57735027]
 [ 0.          0.70710678  0.70710678]
 [ 0.81649658 -0.40824829  0.40824829]]
Matrix Product(USV): [[ 1.00000000e+00  1.00000000e+00  2.52368982e-17]
 [-1.00000000e+00  1.27610516e-17  1.00000000e+00]
 [-8.05746207e-17  1.00000000e+00 -1.00000000e+00]
 [ 1.00000000e+00  1.00000000e+00 -1.00000000e+00]]
Time for computation 0.0010106563568115234


In [274]:
st = time.time()
A5=np.array([[0,1,1],[0,1,0],[1,1,0],[0,1,0],[1,0,1]])
U5,S5,V5=np.linalg.svd(A5)

Sigma5 = sigma(S5,U5.shape[1],V5.shape[1])

D5=np.dot(np.dot(U5,Sigma5),V5)

et = time.time()
print("Matrix(U):",U5)

print("Matrix(S):",Sigma5)

print("Matrix(V):",V5)

print("Matrix Product(USV):",D5)

print("Time for computation",et-st)

Matrix(U): [[-5.47722558e-01 -4.11856119e-16  7.07106781e-01 -1.32058463e-01
  -4.27271064e-01]
 [-3.65148372e-01  4.08248290e-01  1.60009123e-16 -5.43516408e-01
   6.36073827e-01]
 [-5.47722558e-01  1.63994382e-16 -7.07106781e-01 -1.32058463e-01
  -4.27271064e-01]
 [-3.65148372e-01  4.08248290e-01  2.13371358e-16  8.07633333e-01
   2.18468301e-01]
 [-3.65148372e-01 -8.16496581e-01 -9.35206447e-17  1.32058463e-01
   4.27271064e-01]]
Matrix(S): [[2.23606798 0.         0.        ]
 [0.         1.41421356 0.        ]
 [0.         0.         1.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]]
Matrix(V): [[-4.08248290e-01 -8.16496581e-01 -4.08248290e-01]
 [-5.77350269e-01  5.77350269e-01 -5.77350269e-01]
 [-7.07106781e-01  3.33066907e-16  7.07106781e-01]]
Matrix Product(USV): [[-1.92849653e-16  1.00000000e+00  1.00000000e+00]
 [-1.95456893e-16  1.00000000e+00 -6.86829266e-16]
 [ 1.00000000e+00  1.00000000e+00 -4.43466881e-16]
 [-1.94853750e-16  1.00000000e+0

# Experimental Scripts

In [231]:
import numpy as np
import array_to_latex as a2l
# Steepest Descent method
def conjugate_gradient(A, b, x0, tol=0.001, max_iter=1000):
    """
    Solve the linear system Ax = b using Conjugate Gradient method.

    Parameters:
        A (numpy.ndarray): Coefficient matrix.
        b (numpy.ndarray): Right-hand side vector.
        x0 (numpy.ndarray): Initial guess for the solution.
        tol (float): Tolerance for convergence.
        max_iter (int): Maximum number of iterations.

    Returns:
        x (numpy.ndarray): Solution vector.
        num_iter (int): Number of iterations performed.
    """
    fm = '{:1.1e}'
    float_formatter = fm.format
    np.set_printoptions(formatter={'float_kind':float_formatter})
    r = b - np.dot(A, x0)
    v = r
    x = x0
    rsold = np.dot(r, r)
    for i in range(max_iter):
        print("Iteration",i,":")
        print("r=")
        a2l.to_ltx(r, frmt = fm, arraytype = 'bmatrix')
        Av = np.dot(A, v)
        print("Av=")
        a2l.to_ltx(Av, frmt = fm, arraytype = 'bmatrix')
        t = np.dot(r,v) / np.dot(v, Av)
        print("t=",fm.format(t))
        x = x + t * v
        print("x=")
        a2l.to_ltx(x, frmt = fm, arraytype = 'bmatrix')
        r = b - np.dot(A,x)
        v = r
        print("r=")
        a2l.to_ltx(r, frmt = fm, arraytype = 'bmatrix')
        rsnew = np.dot(r, r)
        print("|r|^2=",fm.format(rsnew))
        if np.sqrt(rsnew) < tol:
            break
    return x, i+1

# Solving by steepest descent method
A = np.array([[1.00,0.50],[0.50,0.33]])
b = np.array([0.24,0.13])
x0 = np.array([0,0])
solution, iterations = conjugate_gradient(A, b, x0)
print("Solution:", solution)
print("Number of iterations:", iterations)


Iteration 0 :
r=
\begin{bmatrix}
  2.4\times 10^{-01} &  1.3\times 10^{-01}
\end{bmatrix}
Av=
\begin{bmatrix}
  3.0\times 10^{-01} &  1.6\times 10^{-01}
\end{bmatrix}
t= 7.9e-01
x=
\begin{bmatrix}
  1.9\times 10^{-01} &  1.0\times 10^{-01}
\end{bmatrix}
r=
\begin{bmatrix}
 -7.6\times 10^{-04} &  1.4\times 10^{-03}
\end{bmatrix}
|r|^2= 2.6e-06
Iteration 1 :
r=
\begin{bmatrix}
 -7.6\times 10^{-04} &  1.4\times 10^{-03}
\end{bmatrix}
Av=
\begin{bmatrix}
 -5.9\times 10^{-05} &  8.3\times 10^{-05}
\end{bmatrix}
t= 1.6e+01
x=
\begin{bmatrix}
  1.8\times 10^{-01} &  1.2\times 10^{-01}
\end{bmatrix}
r=
\begin{bmatrix}
  1.7\times 10^{-04} &  9.0\times 10^{-05}
\end{bmatrix}
|r|^2= 3.6e-08
Solution: [1.8e-01 1.2e-01]
Number of iterations: 2
